Setup to run train with in a notebook

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("../")
os.getcwd()

'/home/mwirth/projects/sound-seperation'

In [36]:
from hydra import compose, initialize
from hydra.utils import instantiate

In [37]:
with initialize(version_base="1.3", config_path="../configs/"):
    cfg = compose(config_name="train.yaml", return_hydra_config=True, #overrides=["experiment=HSN/HSN_Wav2vec2.yaml"]
                  )
cfg.datamodule.hf_dataset.cache_dir = "/home/mwirth/projects/sound-seperation/data/DBD-research-group/BirdSet/XCM"

In [38]:
dm = instantiate(cfg["datamodule"], _recursive_=False)

Filter:   0%|          | 0/81025 [00:00<?, ? examples/s]

In [42]:
dm.hf_train_split[:5]

{'audio': [{'bytes': None,
   'path': '/home/mwirth/projects/sound-seperation/notebooks/moritz/data/DBD-research-group/BirdSet/XCM/downloads/extracted/2cd960300035e5d02e155dda3ccb2fd1a80a07b2c6ebd3e1f8c4c4fbcceca646/data/xeno-canto/north_america/aix_sponsa/XC210720.ogg'},
  {'bytes': None,
   'path': '/home/mwirth/projects/sound-seperation/notebooks/moritz/data/DBD-research-group/BirdSet/XCM/downloads/extracted/a480c6400c5a329b34fea8cc3d22dd76b179556fa3ba0267178b5152337ecbda/data/xeno-canto/europe/aegithalos_caudatus/XC707394.ogg'},
  {'bytes': None,
   'path': '/home/mwirth/projects/sound-seperation/notebooks/moritz/data/DBD-research-group/BirdSet/XCM/downloads/extracted/93fdbe6fd0ac5cac86d84d345ea1a6d018c88c44430fcf35560182ba734ae63a/data/xeno-canto/south_america/dendrocincla_fuliginosa/XC248437.ogg'},
  {'bytes': None,
   'path': '/home/mwirth/projects/sound-seperation/notebooks/moritz/data/DBD-research-group/BirdSet/XCM/downloads/extracted/3f1811816345a6a726448ea098df08e86cbc845c44

In [43]:
dm.hf_valid_split[:5]

{'audio': [{'bytes': None,
   'path': '/home/mwirth/projects/sound-seperation/notebooks/moritz/data/DBD-research-group/BirdSet/XCM/downloads/extracted/be789baf1d713bc8f6d3e99694f891388c50d7c6a4c642532aa6c05901c33c95/data/xeno-canto/north_america/larus_delawarensis/XC71564.ogg'},
  {'bytes': None,
   'path': '/home/mwirth/projects/sound-seperation/notebooks/moritz/data/DBD-research-group/BirdSet/XCM/downloads/extracted/50af9717b79ff9b0982abccb6c8f11b4dbc0e36a2ca44b3e226c8f0f7735e7af/data/xeno-canto/north_america/melospiza_lincolnii/XC539278.ogg'},
  {'bytes': None,
   'path': '/home/mwirth/projects/sound-seperation/notebooks/moritz/data/DBD-research-group/BirdSet/XCM/downloads/extracted/5076c2c75a73c4986753d23b9f50b5937d03d67cbc441277eebb07eea1f59f34/data/xeno-canto/europe/muscicapa_striata/XC562627.ogg'},
  {'bytes': None,
   'path': '/home/mwirth/projects/sound-seperation/notebooks/moritz/data/DBD-research-group/BirdSet/XCM/downloads/extracted/5e8b3e3fa1e97209ea7091ad3440bfbb584a6899c